# sample table

In [1]:
import seaborn as sb

df = sb.load_dataset('tips')
conn = "sqlite:///example"  
df.to_sql(
            'tips',            
            conn,                
            if_exists='replace'
           )
print(df)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


# DENSE_RANK()

DENSE_RANK() is a window function in SQL that assigns a rank to each row within a partition of a result set. It is similar to the RANK() function, but DENSE_RANK() does not leave gaps in the ranking sequence when there are ties.

For example, if two rows are tied for rank 1, the next rank assigned will be 2, not 3. This function is useful for scenarios where you want to rank items without skipping numbers.



In [27]:
import pandas as pd
query_result = pd.read_sql("""SELECT total_bill, tip, sex, DENSE_RANK() OVER (ORDER BY total_bill DESC) AS Rank
            FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex
            FROM tips)""", con=conn)
query_result.head()

,total_bill,tip,sex,Rank
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,2
3,48,9.00,Male,2
4,45,3.50,Male,3


# RANK()
 
RANK() is a window function in SQL that assigns a unique rank to each row within a partition of a result set. Unlike DENSE_RANK(), the RANK() function leaves gaps in the ranking sequence when there are ties. 

For example, if two rows are tied for rank 1, the next rank assigned will be 3, not 2. This function is useful for scenarios where you want to rank items and acknowledge the gaps created by ties.


In [28]:
import pandas as pd
query_result = pd.read_sql("""SELECT total_bill, tip, sex, RANK() OVER (ORDER BY total_bill DESC) AS Rank
            FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex
            FROM tips)""", con=conn)
query_result.head()

,total_bill,tip,sex,Rank
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,2
3,48,9.00,Male,2
4,45,3.50,Male,5


# AS

Using SELECT inside another select

In [5]:
import pandas as pd

query_result = pd.read_sql("""SELECT * FROM tips WHERE total_bill IN (SELECT total_bill FROM (SELECT DISTINCT total_bill FROM tips LIMIT 5) AS unique_bills)""", con=conn)
query_result.head()

,index,total_bill,tip,sex,smoker,day,time,size
0,0,16.99,1.01,Female,No,Sun,Dinner,2
1,1,10.34,1.66,Male,No,Sun,Dinner,3
2,2,21.01,3.50,Male,No,Sun,Dinner,3
3,3,23.68,3.31,Male,No,Sun,Dinner,2
4,4,24.59,3.61,Female,No,Sun,Dinner,4


# MINUS OR EXCEPT


The `MINUS` operator in SQL is used to return all rows in the first SELECT statement that are not returned by the second SELECT statement. It is supported in some SQL dialects like Oracle. In other databases like PostgreSQL, the equivalent is `EXCEPT`.


In [15]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35
5       44.30


In [16]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35


In [14]:
import pandas as pd

# Example: Get all unique total_bill values in tips that are not among the top 5 highest total_bill values
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)
EXCEPT
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)
""", con=conn)
query_result.head()


,total_bill
0,44.3


# UNION

The UNION operator in SQL is used to combine the result sets of two or more SELECT statements.
It returns all distinct rows selected by either query.
Each SELECT statement within the UNION must have the same number of columns, and the columns must have compatible data types.

Example:

SELECT column1, column2 FROM table1

UNION

SELECT column1, column2 FROM table2;


In [17]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35
5       44.30


In [18]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35


In [20]:
import pandas as pd

# Example: Get all unique total_bill values in tips that are not among the top 5 highest total_bill values
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)
UNION
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)
""", con=conn)
query_result

,total_bill
0,44.30
1,45.35
2,48.17
3,48.27
4,48.33
5,50.81


# UNION ALL

The UNION ALL operator in SQL is used to combine the result sets of two or more SELECT statements.
Unlike UNION, UNION ALL does not remove duplicate rows; it returns all rows from each SELECT statement, including duplicates.

Example:

SELECT column1, column2 FROM table1

UNION ALL

SELECT column1, column2 FROM table2;

This will return all rows from both table1 and table2, including any duplicates.


In [21]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35
5       44.30


In [22]:
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)""", conn)
print(query_result)

   total_bill
0       50.81
1       48.33
2       48.27
3       48.17
4       45.35


In [23]:
import pandas as pd

# Example: Get all unique total_bill values in tips that are not among the top 5 highest total_bill values
query_result = pd.read_sql("""
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 6
)
UNION ALL
SELECT total_bill FROM (
    SELECT DISTINCT total_bill FROM tips ORDER BY total_bill DESC LIMIT 5
)
""", con=conn)
query_result

,total_bill
0,50.81
1,48.33
2,48.27
3,48.17
4,45.35
5,44.30
6,50.81
7,48.33
8,48.27
9,48.17


# windows

Window functions in SQL are used to perform calculations across a set of table rows that are somehow related to the current row. 
Unlike aggregate functions, window functions do not cause rows to become grouped into a single output row; the rows retain their separate identities.
 
Common window functions include **RANK(), DENSE_RANK(), ROW_NUMBER(), SUM(), AVG(), etc.**
 
The general syntax for a window function is:
 
    function_name(column) OVER (

       PARTITION BY column1
       ORDER BY column2

    )
 
 - PARTITION BY divides the result set into partitions to which the window function is applied.
 - ORDER BY defines the logical order of the rows within each partition.
 
 Example:

    SELECT

       total_bill,

       RANK() OVER (ORDER BY total_bill DESC) AS bill_rank

    FROM tips;
 
This assigns a rank to each row based on the total_bill value in descending order.



In [2]:
import pandas as pd
query_result = pd.read_sql("""SELECT total_bill, tip, sex, RANK() OVER (ORDER BY total_bill DESC) AS Rank
            FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex
            FROM tips)""", con=conn)
query_result.head()

,total_bill,tip,sex,Rank
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,2
3,48,9.00,Male,2
4,45,3.50,Male,5


# Row_number


The ROW_NUMBER() window function assigns a unique sequential integer to rows within a result set, starting at 1 for the first row in each partition.
 

In [3]:
import pandas as pd
query_result = pd.read_sql("""
    SELECT total_bill, tip, sex, ROW_NUMBER() OVER (ORDER BY total_bill DESC) AS row_num
    FROM (SELECT CAST(total_bill AS INTEGER) AS total_bill, tip, sex FROM tips)
""", con=conn)
query_result.head()

,total_bill,tip,sex,row_num
0,50,10.00,Male,1
1,48,6.73,Male,2
2,48,5.00,Male,3
3,48,9.00,Male,4
4,45,3.50,Male,5
